<a href="https://colab.research.google.com/github/tensorflow-project/FineTuning/blob/main/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/tensorflow-project/FineTuning
%cd FineTuning

Cloning into 'FineTuning'...
remote: Enumerating objects: 379, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 379 (delta 16), reused 5 (delta 5), pack-reused 361
Receiving objects: 100% (379/379), 84.59 MiB | 25.12 MiB/s, done.
Resolving deltas: 100% (280/280), done.
/content/FineTuning


In [2]:
from numpy import dot
from numpy.linalg import norm
import os
from google.colab import drive
from PIL import Image
import textual_inversion as txt
#from textual_inversion import train_ds

  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


ImportError: ignored

In [ ]:
sticker_embedding = []
cosine_similarity = []
broccoli = txt.get_embedding("broccoli")
cosine_similarity.append(txt.cosine_sim(broccoli, txt.get_embedding(placeholder_token)))

In [ ]:
epochs = 5
epoch_num = list(range(epochs+1))
txt.training(epoch=epochs, model=txt.stable_diffusion, data= txt.train_ds)

In [ ]:
plt.plot(epoch_num, cosine_similarity)
plt.xlabel("Number of Epochs")
plt.ylabel("Cosine Similarity")
plt.title("Cosine Similarity between the basis and the new concept")
plt.show()

In [ ]:
drive.mount("/content/drive")
path = '/content/drive/MyDrive/angryweeeights.npy'

In [ ]:
###get the weights of the text encoder and save the to google drive
text_encoder_weights = np.array(stable_diffusion.text_encoder.get_weights())

### Save the weights array to a file on your Google Drive
np.save(path, text_encoder_weights)

In [ ]:
###load the array of the weights of the text encoder from the last training from google drive
text_encoder_weights = np.load(path, allow_pickle=True)

### Set the weights of the text encoder
stable_diffusion.text_encoder.set_weights(text_encoder_weights)

In [ ]:
import shutil
drive_folder = '/content/drive/MyDrive/angrybildeeer/'

### get the number, we stopped the last time to name our pictures, to ensure each picture gets a different name
i_file = os.path.join(drive_folder, 'i.txt')
if os.path.isfile(i_file):
    with open(i_file, 'r') as f:
        i = int(f.read())
else:
    i = 0

for j in range(10):
    generated = stable_diffusion.text_to_image(
    f" a angry {placeholder_token}.", batch_size=1,  num_steps=25 )
    broc = generated[0]

    ### convert the array generated from our stable diffusion model into a picture
    broc = Image.fromarray(broc, mode='RGB')

    broc.save(f'image_{i}.jpg')

    ### save the picture to google drive
    local_path = f'image_{i}.jpg'
    drive_path = os.path.join(drive_folder, f'image_{i}.jpg')  # Use f-string to include variable in file name
    shutil.copy(local_path, drive_path)

    ### store the value of i in the file, to ensure no picture will have the same name
    i += 1
    with open(i_file, 'w') as f:
        f.write(str(i))